In [18]:
import pandas as pd
import numpy as np  
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
hotel_city_chanel_combin_extract  = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\other\\hotel_city_chanel_combin_extract.csv')
hotel_city_chanel_combin_extract.dropna(inplace=True)
hotel_city_chanel_combin_extract

In [ ]:
print(hotel_city_chanel_combin_extract['Canal_ID'].nunique(), 
        hotel_city_chanel_combin_extract['Cidade_ID'].nunique() ,
            hotel_city_chanel_combin_extract['Hotel_ID'].nunique() )

In [5]:
def get_unseen_hotels(df, canal_id):
    # Ensure canal_id is an integer
    if not isinstance(canal_id, int):
        raise ValueError(f"Expected an integer for canal_id, but got {type(canal_id)}.")

    # Get the Cidade_IDs corresponding to the given Canal_ID
    cidade_ids = df.loc[df['Canal_ID'] == canal_id, 'Cidade_ID'].unique()

    # Get all Hotel_IDs in the corresponding Cidade_IDs
    hotels_in_cidade = df.loc[df['Cidade_ID'].isin(cidade_ids), 'Hotel_ID'].unique()

    # Get the Hotel_IDs already associated with the given Canal_ID
    seen_hotels = df.loc[df['Canal_ID'] == canal_id, 'Hotel_ID'].unique()

    # Filter out seen hotels
    unseen_hotels = [hotel for hotel in hotels_in_cidade if hotel not in seen_hotels]

    # Return unseen hotels as a DataFrame column, ensuring uniqueness
    unseen_hotels_df = pd.DataFrame({'Unseen_Hotel_ID': unseen_hotels}).drop_duplicates()

    return unseen_hotels_df

In [ ]:
get_unseen_hotels(hotel_city_chanel_combin_extract, 1245)

In [8]:
data_lake_prd_314410_cz_hoteis = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\lookups\\data-lake-prd-314410.cz.hoteis.csv')

data_lake_prd_314410_cz_hoteis = data_lake_prd_314410_cz_hoteis[['Hotel_ID', 'Estrelas', 'Quartos', 'CategoriaHotel', 'Moeda', 'Cidade_ID']]

data_lake_prd_314410_cz_cidades = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\lookups\\data-lake-prd-314410.cz.cidades.csv')

data_lake_prd_314410_cz_cidades = data_lake_prd_314410_cz_cidades[['Cidade_ID', 'Pais']]

merged_data = pd.merge(data_lake_prd_314410_cz_hoteis, 
                       data_lake_prd_314410_cz_cidades, 
                       on='Cidade_ID', 
                       how='left')

merged_data.dropna(subset=['Pais'], inplace=True)

merged_data.drop(columns=['Cidade_ID'], inplace=True)


In [ ]:
data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\other\\data-lake-prd-314410.cz.pull-pesquisas_average_price_hotel_currency.csv')
data_lake_prd_314410_cz_moedas = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\lookups\\data-lake-prd-314410.cz.moedas.csv')

data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency = pd.merge(data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency, 
                     data_lake_prd_314410_cz_moedas[['Moeda_ID', 'Cotacao_USD']], 
                     on='Moeda_ID', 
                     how='left')


data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency['weighted_mean_diaria_usd'] = \
    data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency['weighted_mean_diaria'] * \
        data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency['Cotacao_USD']

merged_data = pd.merge(merged_data, 
                     data_lake_prd_314410_cz_pull_pesquisas_average_price_hotel_currency[['Hotel_ID', 'Cotacao_USD', 'weighted_mean_diaria_usd']], 
                     on='Hotel_ID', 
                     how='left')

data_lake_prd_314410_cz_pull_pesquisas_estadia_x_reservas_volume = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\other\\data-lake-prd-314410.cz.pull-pesquisas_estadia_x_reservas_volume.csv')

merged_data = pd.merge(merged_data, 
                     data_lake_prd_314410_cz_pull_pesquisas_estadia_x_reservas_volume, 
                     on='Hotel_ID', 
                     how='left')

data_lake_prd_314410_cz_pull_pesquisas_estadia_x_reservas_x_price_receitas = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\other\\data-lake-prd-314410.cz.pull-pesquisas_estadia_x_reservas_x_price_receitas.csv')
data_lake_prd_314410_cz_pull_pesquisas_estadia_x_reservas_x_price_receitas

merged_data = pd.merge(merged_data, 
                     data_lake_prd_314410_cz_pull_pesquisas_estadia_x_reservas_x_price_receitas, 
                     on='Hotel_ID', 
                     how='left')

merged_data['total_reservas_estadia_diaria'] = \
    merged_data['total_reservas_estadia_diaria'] * \
        merged_data['Cotacao_USD']

merged_data.drop(columns=['Moeda'], inplace=True)

merged_data.drop(columns=['Cotacao_USD'], inplace=True)

merged_data.dropna(inplace=True)

merged_data

In [ ]:
merged_data.drop(columns=['Pais', 'CategoriaHotel'], inplace=True)

In [ ]:
merged_data

In [ ]:
merged_data

In [50]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

def rank_suggested_hotels(df_hotels, df_characteristics, canal_id, suggested_hotels_df):
    # Scale features
    scaler = StandardScaler()
    features_to_scale = ['Estrelas', 'Quartos', 'weighted_mean_diaria_usd', 
                         'total_reservas_estadia', 'total_reservas_estadia_diaria']
    characteristics_scaled = df_characteristics.copy()
    characteristics_scaled[features_to_scale] = scaler.fit_transform(
        characteristics_scaled[features_to_scale]
    )
    
    # Get the original hotels for the given Canal_ID
    original_hotels = df_hotels[df_hotels['Canal_ID'] == canal_id]
    original_characteristics = characteristics_scaled[
        characteristics_scaled['Hotel_ID'].isin(original_hotels['Hotel_ID'])
    ]
    
    # Join suggested hotels with their characteristics
    suggested_hotels = pd.merge(
        suggested_hotels_df,
        characteristics_scaled,
        left_on='Unseen_Hotel_ID',
        right_on='Hotel_ID',
        how='left'
    )
    
    # Add city information to suggested hotels
    suggested_hotels = pd.merge(
        suggested_hotels,
        df_hotels[['Hotel_ID', 'Cidade_ID']].drop_duplicates(),
        left_on='Unseen_Hotel_ID',
        right_on='Hotel_ID',
        suffixes=('', '_city'),
        how='left'
    )
    
    ranked_results = []
    
    for city_id in suggested_hotels['Cidade_ID'].unique():
        # Filter for hotels in this city
        city_suggested_hotels = suggested_hotels[suggested_hotels['Cidade_ID'] == city_id]
        city_original_hotels = original_characteristics[
            original_characteristics['Hotel_ID'].isin(
                original_hotels[original_hotels['Cidade_ID'] == city_id]['Hotel_ID']
            )
        ]
        
        if city_original_hotels.empty or city_suggested_hotels.empty:
            continue
        
        # Handle missing values
        city_suggested_hotels[features_to_scale] = city_suggested_hotels[features_to_scale].fillna(0)
        city_original_hotels[features_to_scale] = city_original_hotels[features_to_scale].fillna(0)
        
        # Extract feature matrices
        suggested_features = city_suggested_hotels[features_to_scale].values
        original_features = city_original_hotels[features_to_scale].values
        
        # Calculate cosine similarity
        similarity_matrix = cosine_similarity(suggested_features, original_features)
        
        # Rank suggested hotels by their maximum similarity to original hotels
        city_suggested_hotels['Max_Similarity'] = similarity_matrix.max(axis=1)
        city_ranked = city_suggested_hotels.sort_values(
            by='Max_Similarity', ascending=False
        ).reset_index(drop=True)
        
        # Exclude original hotels from the ranking (sanity check)
        city_ranked = city_ranked[~city_ranked['Unseen_Hotel_ID'].isin(original_hotels['Hotel_ID'])]
        
        ranked_results.append(city_ranked)
    
    # Combine ranked results for all cities
    final_ranked = pd.concat(ranked_results, ignore_index=True) if ranked_results else pd.DataFrame()
    return final_ranked


In [ ]:
counts = hotel_city_chanel_combin_extract.groupby('Canal_ID').size().reset_index(name='Counts').sort_values(by='Counts', ascending=True)
counts[counts['Counts'] >100]

In [41]:
suggested_hotels = get_unseen_hotels(hotel_city_chanel_combin_extract, 1124)

In [ ]:
suggested_hotels

In [ ]:
# Example usage
ranked_hotels = rank_suggested_hotels(
    hotel_city_chanel_combin_extract,
    merged_data,
    1124,
    suggested_hotels
)
print(ranked_hotels.head())


In [ ]:
ranked_hotels.groupby('Unseen_Hotel_ID').size().reset_index(name='Counts').sort_values(by='Counts', ascending=False)

In [ ]:
ranked_hotels

In [ ]:
ranked_hotels[ranked_hotels['Unseen_Hotel_ID'] == 13992.0]

In [ ]:
suggested_hotels[suggested_hotels['Unseen_Hotel_ID'] == 19974.0]

In [ ]:
hotel_city_chanel_combin_extract[hotel_city_chanel_combin_extract['Hotel_ID'] == 19974.0]